In [1]:
#!/usr/bin/env python3

import os
import pathlib
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from config import DATALAKE


raw = 'raw_data_covid19_version-' + datetime.now().strftime('%Y-%m-%d')
output_sg = os.path.join(DATALAKE, raw, 'data-notificacao_sindrome_gripal')

url = 'https://opendatasus.saude.gov.br/dataset/'

list_endpoints = [
    'notificacoes-de-sindrome-gripal-leve-2020',
    'notificacoes-de-sindrome-gripal-leve-2021',
    'notificacoes-de-sindrome-gripal-leve-2022',
]

ufs = [
    'AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO',
    'AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI',
    'RN', 'SE', 'DF', 'GO', 'MT', 'MS', 'ES',
    'MG', 'RJ', 'SP', 'PR', 'RS', 'SC'
]

for endpoint in list_endpoints:
    part_parth = endpoint.replace('notificacoes-de-', '')
    folder_dataset = os.path.join(output_sg, part_parth)
    pathlib.Path(folder_dataset).mkdir(parents=True, exist_ok=True)

    r = requests.get(os.path.join(url, endpoint))
    soup = BeautifulSoup(r.text, 'html.parser')
    tag_a = soup.findAll('a')
    list_uf_text = list(map(lambda x: 'Dados ' + x, ufs))
    data_url = {}

    for tag in tag_a:
        string = tag.text.lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')[0:8]

        for string_uf in list_uf_text:
            if string_uf == string:

                href = tag['href']
                data_url[string] = href.lstrip('/dataset/')

    for csv_url in data_url.values():
        r = requests.get(os.path.join(url, csv_url))
        soup = BeautifulSoup(r.text, 'html.parser')
        tag_a = soup.findAll('a')

        for tag in tag_a:
            if tag['href'].endswith('.csv'):
                file_csv = requests.get(tag['href'], stream=True)

                with open(os.path.join(folder_dataset, tag.text), 'wb') as f, tqdm(
                    desc=tag.text,
                    total=int(file_csv.headers['Content-Length']),
                    unit='iB',
                    unit_scale=True,
                    unit_divisor=1024
                ) as bar:
                    for content in file_csv.iter_content(chunk_size=1024):
                        size = f.write(content)
                        bar.update(size)


UF-AC - Lote 1: 100%|██████████| 47.8M/47.8M [04:17<00:00, 194kiB/s]   
UF-AL - Lote 1: 100%|██████████| 132M/132M [05:12<00:00, 444kiB/s]     
UF-AM - Lote 1: 100%|██████████| 183M/183M [00:27<00:00, 7.11MiB/s] 
UF-AP - Lote 1: 100%|██████████| 42.3M/42.3M [00:07<00:00, 5.64MiB/s]
UF-BA - Lote 1: 100%|██████████| 353M/353M [01:16<00:00, 4.86MiB/s]   
UF-BA - Lote 2: 100%|██████████| 285M/285M [04:42<00:00, 1.06MiB/s]   
UF-CE - Lote 1: 100%|██████████| 336M/336M [00:38<00:00, 9.13MiB/s]   
UF-DF - Lote 1: 100%|██████████| 149M/149M [00:16<00:00, 9.28MiB/s] 
UF-ES - Lote 1: 100%|██████████| 136M/136M [02:05<00:00, 1.13MiB/s]   
UF-GO - Lote 1: 100%|██████████| 361M/361M [01:08<00:00, 5.53MiB/s]    
UF-GO - Lote 2: 100%|██████████| 10.5M/10.5M [00:02<00:00, 3.82MiB/s]
UF-MA - Lote 1: 100%|██████████| 200M/200M [06:09<00:00, 569kiB/s]     
UF-MG - Lote 1: 100%|██████████| 358M/358M [02:51<00:00, 2.19MiB/s]   
UF-MG - Lote 2: 100%|██████████| 357M/357M [07:29<00:00, 833kiB/s]    
UF-MG - 

ChunkedEncodingError: ("Connection broken: ConnectionResetError(54, 'Connection reset by peer')", ConnectionResetError(54, 'Connection reset by peer'))